<a href="https://colab.research.google.com/github/MasaAsami/LTsandox/blob/main/notebooks/sample_nber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Colabでseleniumをつかう
- 動的サイトのデータを取得したい
- ただし、Google Colabのアップデートにより、インストールエラーになる。
- とりあえず、以下の呪文を最初にいれる
https://github.com/kaliiiiiiiiii/Selenium-Profiles/issues/10

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install -q selenium

Executing: /tmp/apt-key-gpghome.YbCP80CXJm/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.6ifLSo6VIm/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.v71AEpindE/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [2]:
import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

import pandas as pd

In [3]:
#@title ブラウザをheadlessモード実行
options = Options()
options.add_argument("--headless") # or use pyvirtualdiplay
options.add_argument("--no-sandbox") # needed, because colab runs as root
# options.add_argument('--disable-dev-shm-usage') 

options.headless = True

driver = webdriver.Chrome("chromedriver", options=options)

<ipython-input-3-795666fc83f2>:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [4]:
page = 1
URL = f"https://www.nber.org/papers?page={page}&perPage=100&sortBy=public_date"

driver.get(URL)
time.sleep(3)
print(driver.title)

Working Papers | NBER


In [5]:
elems = driver.find_elements(By.CLASS_NAME, "paper-card__paper_number")

# for elem in elems:
#     print("論文ID : ",elem.text)

In [6]:
paper_ids = [elem.text for elem in elems]
# TODO ここで真に必要なIDに絞る
paper_ids = paper_ids[:30]
paper_ids

['30990',
 '30991',
 '30993',
 '30883',
 '30984',
 '30988',
 '30989',
 '30983',
 '30985',
 '30986',
 '30987',
 '30960',
 '30961',
 '30963',
 '30964',
 '30965',
 '30976',
 '30977',
 '30978',
 '30979',
 '30980',
 '30981',
 '30972',
 '30973',
 '30974',
 '30962',
 '30959',
 '30970',
 '30975',
 '30966']

In [7]:
%%time
result_dict = {}

for _id in tqdm(paper_ids):
    driver.get(f"https://www.nber.org/papers/w{_id}")
    time.sleep(3)

    authors_list = driver.find_elements(By.CLASS_NAME, "page-header__author-item")
    author_str = ""
    for i, _author in enumerate(authors_list):
        author_str += " " + _author.text if i > 0 else _author.text

    _citation_items = driver.find_elements(By.CLASS_NAME,"page-header__citation-item")
    doi = _citation_items[1].text.split(" ")[1]
    issue_date = _citation_items[2].text.split(" ")[2] + " " + _citation_items[2].text.split(" ")[3]

    abst_str = driver.find_element(By.CLASS_NAME,"page-header__intro-inner").text

    result_dict[_id] = {
        "url" : f"https://www.nber.org/papers/w{_id}",
        "paper_title" : driver.find_element(By.CLASS_NAME, "page-header__title").text,
        "author" : author_str,
        "doi" : doi,
        "issue_date" : issue_date,
        "abst" : abst_str
    }
    
nber_df =  pd.DataFrame(result_dict).T

100%|██████████| 30/30 [02:01<00:00,  4.05s/it]

CPU times: user 1.31 s, sys: 107 ms, total: 1.42 s
Wall time: 2min 1s


In [8]:
from google.colab import files

t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
_now = datetime.datetime.now(JST).strftime('%Y%m%d%H%M%S')

nber_df.to_csv(f'nber_{_now}.csv', index=False)
files.download(f'nber_{_now}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
driver.quit()